In [4]:
from zipfile import ZipFile
import pandas as pd
import math
import numpy as np
import networkx as nx

In [5]:
data_path =  "./data/PBMC-Zheng4k/combined_PBMC-Zheng4k-Net.csv"
label_path =  "./data/PBMC-Zheng4k/label.csv"

In [6]:
data = pd.read_csv(data_path, header=0, index_col=0, sep=',')
y = pd.read_csv(label_path, index_col=0, header=0,sep = ',')

In [7]:
data[1:5]

,0,1,2,3,4,5,6,7,8,9,...,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599
1,13.195904,-2.204400,-1.675026,-1.886662,-0.942392,0.072390,-0.479128,0.513208,-0.575137,0.471893,...,0.023099,-0.050246,-0.039962,0.005300,-0.076916,-45.588795,3.970543,1.361163,-1.195024,-0.320461
2,13.141597,-0.823840,-1.151293,-1.784650,0.735409,2.807315,-0.678284,-0.633671,0.320261,0.147066,...,0.024684,0.013477,-0.005605,-0.019772,0.006352,-104.525590,5.342262,-0.524928,2.017737,-2.167673
3,-5.194341,-2.185114,0.249557,1.718738,-3.887374,-0.972851,0.373556,2.413629,0.191459,-0.398640,...,0.002316,-0.000070,-0.001134,0.000501,0.008088,-57.391735,-0.519332,-0.207988,0.165529,0.743099
4,-4.712175,-1.221622,-3.370716,-0.287806,2.268875,-2.117491,1.200573,-0.181852,-0.910379,-0.403543,...,0.012249,0.013085,-0.010486,0.006607,0.003310,-22.118070,-4.284555,-0.705645,0.378655,-0.319745


In [8]:
y

,Cluster
Barcode,
AAACCTGAGAAGGCCT-1,2
AAACCTGAGACAGACC-1,2
AAACCTGAGATAGTCA-1,2
AAACCTGAGCGCCTCA-1,1
AAACCTGAGGCATGGT-1,1
...,...
TTTGGTTTCGCTAGCG-1,2
TTTGTCACACTTAACG-1,3
TTTGTCACAGGTCCAC-1,3


In [9]:
import scanpy as sc
def Selecting_highly_variable_genes(X, highly_genes):
    adata = sc.AnnData(X)
    adata.var_names_make_unique()
    sc.pp.filter_genes(adata, min_cells=3)
    sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
    sc.pp.log1p(adata)
    adata.raw = adata
    sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5, n_top_genes=highly_genes)
    adata = adata[:, adata.var['highly_variable']].copy()
    data = adata.X

    return data


def binomial_deviance(data,p,n):    
    i=len(n)
    j=len(p)
    A=[]

    for x in range(j):
        sum=0
        for y in range(i):
            sum=sum+data[y,x]*np.log(data[y,x]/(n[y])*p[x]+1e-5)+(n[y]-data[y,x])*np.log((n[y]-data[y,x])/(n[y]*(1-p[x]))+1e-5)
        A.append(sum)
    A=np.array(A)
    return A     


def UMI_cell(data):          
    ni= np.sum(data,axis=1)   
    return ni

def abundant(data):   
    t=data.shape[0]   
    y=np.sum(data,axis=0)   
    ni_total=np.sum(UMI_cell(data))
    return y/ni_total

def selection_gene_ftest():
    adata = pd.read_csv(data_path, header=0, index_col=0, sep=',')
    sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e2, min_counts=-1000)
    print(adata)
    sc.pp.log1p(adata)
    data =adata.X
    return adata

def selection_gene(data,highly_genes):
    ni = UMI_cell(data)
    pai = abundant(data)
    de = binomial_deviance(data, pai, ni)
    data = np.row_stack((de, data))  
    data.T[np.lexsort(data[::-1, :])].T  
      
    data = np.delete(data, 0, axis=0)  
    adata = sc.AnnData(data)
    adata.var_names_make_unique()
    sc.pp.filter_genes(adata, min_cells=3)
    sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
    sc.pp.log1p(adata)
    adata.raw = adata  # save the raw data
    data = adata.X
    data = data[:, 0:highly_genes]
    return data

def geneNet():
    # 读取细胞-基因表达矩阵
    expr_df = pd.read_csv("./data/PBMC-Zheng4k/data.csv")
    network_files = [
        "./data/PBMC-Zheng4k/sm_1_1_edges.csv",
        "./data/PBMC-Zheng4k/sm_1_2_edges.csv",
        "./data/PBMC-Zheng4k/sm_1_3_edges.csv",
        "./data/PBMC-Zheng4k/sm_1_4_edges.csv",
        "./data/PBMC-Zheng4k/sm_1_5_edges.csv",
        "./data/PBMC-Zheng4k/sm_1_6_edges.csv"
    ]
    feature_matrix = np.zeros((expr_df.shape[0], len(network_files)))
    coexp_networks = []
    for i, file_path in enumerate(network_files):
        edges_df = pd.read_csv(file_path)
        G = nx.Graph()
        for _, row in edges_df.iterrows():
            G.add_edge(row['from'], row['to'], weight=row['weight'])
        coexp_networks.append(G)
    
    # 提取加权平均表达值作为特征
    for i, net in enumerate(coexp_networks):
        communities = nx.algorithms.community.louvain_communities(net)
        weighted_module_expr = np.zeros((expr_df.shape[0], len(communities)))
        for c_idx, community in enumerate(communities):
            community = [gene for gene in community if not pd.isnull(gene)]
    
            # 计算模块内基因的加权平均权重
            gene_weights = {gene: sum(net[gene][neighbor]['weight'] for neighbor in net.neighbors(gene)) for gene in
                            community}
    
            # 加权平均表达值
            for cell_id, expr_vec in expr_df.iterrows():
                weighted_avg_expr = sum(expr_vec.loc[gene] * gene_weights.get(gene, 0) for gene in community) / sum(
                    gene_weights.values())
    
                weighted_module_expr[cell_id, c_idx] = weighted_avg_expr
    
        # 取每个模块的加权平均表达值的最大值作为特征（示例中直接取所有模块）
        feature_matrix[:, i] = weighted_module_expr.max(axis=1)
        feature_matrix *= 10 # 将所有网络的特征矩阵扩大10倍
        expr_df = expr_df.apply(pd.to_numeric, errors='coerce')
        adata = sc.AnnData(expr_df)
        adata.obsm["coexpression_features"] = feature_matrix
        return  adata.obsm["coexpression_features"]

# def reshapeX(data):
#     data = np.array(data).astype('float32')
#     data = data[:, 0:1600]

    # # # 拼接网络+降维
    # # coexpression_features = geneNet()
    # # data = np.hstack([data, coexpression_features])
    # # print("测试da",data.shape)
    
    # #  # 对 adata.obsm['X_combined'] 进行 PCA 降维
    # # data = sc.pp.pca(data, n_comps=1600, copy=False)
    # # # print("Shape of X_combined:", data.shape)
    # # # # 保存降维后的数据为 CSV 文件
    # # # data = pd.DataFrame(data.obsm['X_pca_combined'], index=data.obs_names)
    # # print("测试data",data.shape)
    
    # (a, b) = data.shape
    # X = []
    # tmp = data[0, :]
    # print(len(tmp))
    # for i in range(a):
    #     tmp = data[i,:]
    #     tmp = tmp.reshape(( 1, 40, 40))
    #     X.append(tmp)
    # X = np.array(X)
    # return X
def reshapeX(data):
    
    data = np.array(data).astype('float32')
    
    data = data[:,0:1600]
    (a,b) = data.shape
    
    print(f"Processed data shape: {data.shape}")

    X = []
    for i in range(a):
        tmp = data[i,:]
        sum = np.zeros((b,),dtype='float32')
        for j in range(b):
            for k in range(b):
                sum[j] = sum[j]+abs(tmp[j]-tmp[k])
        
        ### 缩放点积模型
        c = np.dot(tmp,tmp)/math.sqrt(b)
        d = np.dot(sum/(b-1),tmp)/math.sqrt(b)
        # alpha = math.exp(c)/(math.exp(c)+math.exp(d))
        # beta = math.exp(d)/(math.exp(c)+math.exp(d))
        max_val = max(c, d)
        exp_c = math.exp(c - max_val)
        exp_d = math.exp(d - max_val)
        alpha = exp_c / (exp_c + exp_d)
        beta = exp_d / (exp_c + exp_d)
        
        sum = alpha*tmp+beta*sum/(b-1)

        sum = sum.reshape((1,40,40))
        X.append(sum)
    X = np.array(X)
    return X

import numpy as np

# def reshapeX(data):
#     data = np.array(data, dtype=np.float32)
#     data = data[:, :1600]  # 仅保留前1600个特征
#     a, b = data.shape
#     print(f"Processed data shape: {data.shape}")
    
#     sqrt_b = np.sqrt(b, dtype=np.float32)  # 预计算平方根
#     X = []
    
#     for i in range(a):
#         tmp = data[i, :]
#         # 向量化计算每个元素的绝对差总和
#         diff = np.abs(tmp[:, np.newaxis] - tmp)
#         sum_total = diff.sum(axis=1)
#         sum_mean = sum_total / (b - 1)  # 计算均值
        
#         # 计算缩放点积
#         c = np.dot(tmp, tmp) / sqrt_b
#         d = np.dot(sum_mean, tmp) / sqrt_b
        
#         # 稳定计算alpha和beta
#         max_val = max(c, d)
#         exp_c = np.exp(c - max_val)
#         exp_d = np.exp(d - max_val)
#         alpha = exp_c / (exp_c + exp_d)
#         beta = exp_d / (exp_c + exp_d)
        
#         # 组合结果并重塑形状
#         combined = alpha * tmp + beta * sum_mean
#         X.append(combined.reshape(1, 40, 40))  # 保持输出形状一致
    
#     X = np.concatenate(X, axis=0)  # 合并所有样本
#     return X

# def reshapeX(data):
#     data = np.array(data).astype('float32')
#     data = data[:, 0:1600]
#     a, b = data.shape
#     print(f"Processed data shape: {data.shape}")

#     X = []
#     for i in range(a):
#         tmp = data[i, :]  # Shape (1600,)
        
#         # 向量化计算绝对差总和（全局依赖建模）
#         diff = np.abs(tmp[:, np.newaxis] - tmp[np.newaxis, :])  # Shape (1600, 1600)
#         sum_abs_diff = np.sum(diff, axis=1)  # Shape (1600,)
#         sum_abs_normalized = sum_abs_diff / (b - 1)  # 归一化差异特征

#         # 计算每个位置的注意力权重（动态权重分配）
#         c_j = tmp * tmp  # 原始特征重要性度量
#         d_j = sum_abs_normalized * tmp  # 差异特征重要性度量
        
#         # 使用Softmax生成权重（稳定性优化）
#         max_vals = np.maximum(c_j, d_j)
#         exp_c = np.exp(c_j - max_vals)
#         exp_d = np.exp(d_j - max_vals)
#         alpha_j = exp_c / (exp_c + exp_d)  # 原始特征权重
#         beta_j = exp_d / (exp_c + exp_d)   # 差异特征权重

#         # 单通道融合：加权合并双通道信息
#         fused_feature = alpha_j * tmp + beta_j * sum_abs_normalized  # Shape (1600,)

#         # 重塑为 (1, 40, 40)
#         X_sample = fused_feature.reshape(1, 40, 40)  # 添加通道维度
#         X.append(X_sample)
    
#     X = np.array(X)  # 最终形状 (a, 1, 40, 40)
#     return X

# def reshapeX(data):
#     data = np.array(data).astype('float32')
#     data = data[:, 0:1600]
#     (a, b) = data.shape
#     X = []
#     tmp=data[0,:]
#     print(len(tmp))
#     for i in range(a):
#         tmp=data[i,:]
#         tmp=tmp.reshape((1,40,40))
#         X.append(tmp)
#     X = np.array(X)
#     return X             
                 
def reshapeY(y):
    y = np.array(y)
    y = y-1
    [a,b] = y.shape
    y = y.reshape((a,))
    return y






In [10]:
def preprocessing(a, y, highly_genes):
    #####Three methods
    #data = Selecting_highly_variable_genes(a, highly_genes)
    #data=selection_gene(a,highly_genes)
    # data=selection_gene_ftest()
    adata = pd.read_csv(data_path, header=0, index_col=0, sep=',')
    adata = sc.AnnData(adata.values, var=pd.DataFrame(index=adata.columns))
    data = adata.X
    data = np.array(data).astype('float32')
    data = data[:, 0:1600]
    data.shape
    X = reshapeX(data)
    y = reshapeY(y)
    print(f"X:{X[0:5]}")
    return X, y


In [11]:
# X,y = preprocessing(data.values.astype('float32'),label.values,highly_genes=1601)
data = np.array(data)

y = np.array(y)

print("预处理前的 data 形状:", data.shape)
print("预处理前的 y 形状:", y.shape)

X, y = preprocessing(data, y, highly_genes=1600)

print("预处理后的 X 形状:", X.shape)
print("预处理后的 y 形状:", y.shape)

预处理前的 data 形状: (4340, 1600)
预处理前的 y 形状: (4340, 1)
Processed data shape: (4340, 1600)
X:[[[[ 1.2204703e+01 -6.7221141e-01 -1.5734848e+00 ...  1.6615609e+00
     1.3688634e-01 -5.2068126e-01]
   [ 4.4164714e-01  9.9607867e-01  4.2533305e-02 ... -4.5110968e-01
     1.3141500e+00  7.9537970e-01]
   [ 1.5308585e+00  6.6992456e-01 -8.5637164e-01 ...  6.2529564e-01
     5.6486613e-01  6.7366332e-01]
   ...
   [ 4.6606801e-02  1.8976774e-02  4.7373813e-02 ...  1.2094936e-03
     2.3282884e-02 -2.2240525e-02]
   [ 4.8110792e-03 -4.6254373e-03  2.7006716e-04 ... -7.9469914e-03
    -1.4880561e-02  1.9759877e-02]
   [-1.4184822e-02 -3.5006194e-03  7.9308189e-03 ... -1.0163307e+00
    -1.0880111e+00 -7.5986989e-02]]]


 [[[ 1.3195904e+01 -2.2044001e+00 -1.6750261e+00 ...  1.2089742e+00
     3.5945922e-01  5.1926965e-01]
   [-4.6099079e-01 -5.0185367e-02 -1.4197189e+00 ...  8.6580056e-01
     1.1032829e+00 -4.5427692e-01]
   [-6.4459634e-01  5.3399757e-02 -2.6203820e-01 ...  1.8305197e-02
    -3.619

In [12]:
import pickle
pickle.dump([X,y],open('./data/PBMC-Zheng4k/PBMC-Zheng4k-Ablation.pkl','wb'))#save to .pkl

In [13]:
# file_path = './data/PBMC-Zheng4k/PBMC-Zheng4k_test.pkl'
# with open(file_path, 'rb') as file:
#     data = pickle.load(file)

# # 假设 data 是一个包含两个元素的列表 [X, y]
# if isinstance(data, list) and len(data) == 2:
#     X, y = data
#     print("X 的维度:", X.shape)
#     print("y 的维度:", y.shape)
# else:
#     print("数据格式不符合预期")

In [14]:
y

array([1, 1, 1, ..., 2, 3, 1])